In [1]:
import cvxpy as cp
import numpy as np
import itertools

from stackelberg_model import *

### Simulations with Single Insertion Errors

In [3]:
n = 10
k = 3
thetas = k*[1] + (n-k)*[0]
m = 2

p1 = thetas
v, p_opt, tight_constraints = get_optimal_strategy(thetas, k, generate_single_insertion_permutations(n, m), verbose=True)

Optimal v*: 2.000
Lexicographically optimal p: [ 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0 ]


In [7]:
n = 10
k = 3
s = k*[1] + (n-k)*[0] # top k

print(s)

for max_shift in range(1, n+1):
    v_out, p_out, tight_constraints = get_optimal_strategy(s, k, generate_single_insertion_permutations(n, max_shift))
    print('shift=', max_shift,':', 'p=', np.round(p_out.tolist(), 3), 'v=', np.round(v_out,3))

[1, 1, 1, 0, 0, 0, 0, 0, 0, 0]
shift= 1 : p= [1.  1.  0.5 0.5 0.  0.  0.  0.  0.  0. ] v= 2.5
shift= 2 : p= [1. 1. 1. 0. 0. 0. 0. 0. 0. 0.] v= 2.0
shift= 3 : p= [1. 1. 1. 0. 0. 0. 0. 0. 0. 0.] v= 2.0
shift= 4 : p= [1. 1. 1. 0. 0. 0. 0. 0. 0. 0.] v= 2.0
shift= 5 : p= [1. 1. 1. 0. 0. 0. 0. 0. 0. 0.] v= 2.0
shift= 6 : p= [1. 1. 1. 0. 0. 0. 0. 0. 0. 0.] v= 2.0
shift= 7 : p= [1. 1. 1. 0. 0. 0. 0. 0. 0. 0.] v= 2.0
shift= 8 : p= [1. 1. 1. 0. 0. 0. 0. 0. 0. 0.] v= 2.0
shift= 9 : p= [1. 1. 1. 0. 0. 0. 0. 0. 0. 0.] v= 2.0
shift= 10 : p= [1. 1. 1. 0. 0. 0. 0. 0. 0. 0.] v= 2.0


In [ ]:
n = 10
k = 2
s = n - np.arange(1, n+1)  # borda count

print(s)

for max_shift in range(1, n+1):
    v_out, p_out, tight_constraints = get_optimal_strategy(s, k, generate_single_insertion_permutations(n, max_shift))
    print('shift=', max_shift,':', 'p=', np.round(p_out.tolist(), 3), 'v=', np.round(v_out,3))

[9 8 7 6 5 4 3 2 1 0]
shift= 1 : p= [1.    0.667 0.333 0.    0.    0.    0.    0.    0.    0.   ] v= 16.333
shift= 2 : p= [1.    0.667 0.333 0.    0.    0.    0.    0.    0.    0.   ] v= 15.667
shift= 3 : p= [0.8 0.6 0.4 0.2 0.  0.  0.  0.  0.  0. ] v= 14.8
shift= 4 : p= [0.82  0.656 0.525 0.    0.    0.    0.    0.    0.    0.   ] v= 14.197
shift= 5 : p= [0.667 0.556 0.463 0.315 0.    0.    0.    0.    0.    0.   ] v= 13.574
shift= 6 : p= [0.621 0.532 0.456 0.391 0.    0.    0.    0.    0.    0.   ] v= 13.037
shift= 7 : p= [0.587 0.514 0.45  0.45  0.    0.    0.    0.    0.    0.   ] v= 12.541
shift= 8 : p= [0.545 0.485 0.485 0.485 0.    0.    0.    0.    0.    0.   ] v= 12.182
shift= 9 : p= [0.5 0.5 0.5 0.5 0.  0.  0.  0.  0.  0. ] v= 12.0
shift= 10 : p= [0.5 0.5 0.5 0.5 0.  0.  0.  0.  0.  0. ] v= 12.0


## Solve with Cost

In [11]:
# Solve an LP with a cost function defined as C(sigma) = C if d(sigma) > cutoff, 0 otherwise
def solve_single_cost_lp(thetas, k, generate_permutations, cutoff, C, max_d, verbose=False):
    permutations, distances = generate_permutations(n, max_d, return_distances=True)
    cost_func = lambda d: C if d > cutoff else 0
    cost = [cost_func(d) for d in distances]

    v_opt, p_opt = get_optimal_strategy_with_costs(thetas, k, permutations, cost, verbose=verbose)
    return v_opt, p_opt

# Solve separate LPs for S1 = {sigma: d(sigma) <= cutoff} and S2 = {sigma: d(sigma) > cutoff}
def solve_separate_lps(thetas, k, generate_permutations, cutoff, max_d):
    n = len(thetas)
    # Permutations S1 and S2
    S = generate_permutations(n, max_d) # set of all perms
    S1 = generate_permutations(n, cutoff) # set of perms with distance <= cutoff
    S2 = [s for s in S if s not in S1] # set of perms with distance > cutoff

    # Solve LP for S1
    v1, p1, _ = get_optimal_strategy(thetas, k, S1)
    # Solve LP for S2
    v2, p2, _ = get_optimal_strategy(thetas, k, S2)
    v,p,_ = get_optimal_strategy(thetas, k, S)

    return v1, p1, v2, p2, v, p

In [12]:
n = 10
k = 1
s = k*[1] + (n-k)*[0]  # borda count
s = s / np.sum(s[:k]) # normalize

max_shift = 8
cutoff = 3

In [13]:
v1, p1, v2, p2, v0, p0 = solve_separate_lps(s, k, generate_single_insertion_permutations, cutoff, max_shift)
print('v1:', v1)
print('p1:', p1)
print('v2:', v2)
print('p2:', p2)
print('v:', v0)
print('p:', p0)

v1: 0.2499999999307309
p1: [0.25 0.25 0.25 0.25 0.   0.   0.   0.   0.   0.  ]
v2: 0.14285714238069763
p2: [1.42857145e-01 1.42857145e-01 5.16045502e-10 0.00000000e+00
 1.42857143e-01 1.42857143e-01 1.42857143e-01 1.42857143e-01
 1.42857143e-01 0.00000000e+00]
v: 0.11111111099967014
p: [0.11111111 0.11111111 0.11111111 0.11111111 0.11111111 0.11111111
 0.11111111 0.11111111 0.11111111 0.        ]


In [14]:
C = 0.04
v,p = solve_single_cost_lp(s, k, generate_single_insertion_permutations, cutoff, C, max_shift)

print('v:', np.round(v,3))
print('p:', p)

v: 0.133
p: [0.13333333 0.13333333 0.13333333 0.13333333 0.09333333 0.09333333
 0.09333333 0.09333333 0.09333333 0.        ]


In [15]:
# find value of lambda such that p1*(lambda) + p0*(1-lambda) = p
def find_lambda(p, p1, p0):
    n = len(p)
    lambda_ = cp.Variable()
    epsilon = 1e-6  # Allowable error
    constraints = [0 <= lambda_, lambda_ <= 1]
    constraints += [cp.abs(p[i] - (lambda_*p1[i] + (1-lambda_)*p0[i])) <= epsilon for i in range(n)]
    problem = cp.Problem(cp.Minimize(0), constraints)
    problem.solve()
    return lambda_.value

In [16]:
find_lambda(p, p1, p0)

array(0.16000014)

In [17]:
C = 0.1
v,p = solve_single_cost_lp(s, k, generate_single_insertion_permutations, cutoff, C, max_shift)

print('v:', np.round(v,3))
print('p:', p)

v: 0.167
p: [0.16666667 0.16666667 0.16666667 0.16666667 0.06666667 0.06666667
 0.06666667 0.06666667 0.06666667 0.        ]


### Simulations with Error at Bounded Kendall-Tau Distance

In [4]:
n = 10

In [6]:
distance_dict = get_all_kendall_tau_distances(n)

In [28]:
k = 1
thetas = k*[1] + (n-k)*[0]

for max_distance in range(1, 15):
    perms = generate_kendall_tau_permutations(n, max_distance, distance_dict=distance_dict)
    v_out, p_out, tight_constraints = get_optimal_strategy(thetas, k, perms, lex_order_p=1) 
    print('D =', max_distance,':', np.round(p_out.tolist(), 3))

D = 1 : [0.5 0.5 0.  0.  0.  0.  0.  0.  0.  0. ]
D = 2 : [0.333 0.333 0.333 0.    0.    0.    0.    0.    0.    0.   ]
D = 3 : [0.25 0.25 0.25 0.25 0.   0.   0.   0.   0.   0.  ]
D = 4 : [0.2 0.2 0.2 0.2 0.2 0.  0.  0.  0.  0. ]
D = 5 : [0.167 0.167 0.167 0.167 0.167 0.167 0.    0.    0.    0.   ]
D = 6 : [0.143 0.143 0.143 0.143 0.143 0.143 0.143 0.    0.    0.   ]
D = 7 : [0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.    0.   ]
D = 8 : [0.111 0.111 0.111 0.111 0.111 0.111 0.111 0.111 0.111 0.   ]
D = 9 : [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
D = 10 : [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
D = 11 : [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
D = 12 : [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
D = 13 : [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
D = 14 : [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]


In [29]:
k = 2
thetas = k*[1] + (n-k)*[0]

for max_distance in range(1, 15):
    perms = generate_kendall_tau_permutations(n, max_distance, distance_dict=distance_dict)
    v_out, p_out, tight_constraints = get_optimal_strategy(thetas, k, perms, lex_order_p=1) 
    print('D =', max_distance,':', np.round(p_out.tolist(), 3))

D = 1 : [1.  0.5 0.5 0.  0.  0.  0.  0.  0.  0. ]
D = 2 : [1. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
D = 3 : [1. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
D = 4 : [0.667 0.667 0.667 0.    0.    0.    0.    0.    0.    0.   ]
D = 5 : [0.667 0.667 0.667 0.    0.    0.    0.    0.    0.    0.   ]
D = 6 : [0.5 0.5 0.5 0.5 0.  0.  0.  0.  0.  0. ]
D = 7 : [0.5 0.5 0.5 0.5 0.  0.  0.  0.  0.  0. ]
D = 8 : [0.4 0.4 0.4 0.4 0.4 0.  0.  0.  0.  0. ]
D = 9 : [0.4 0.4 0.4 0.4 0.4 0.  0.  0.  0.  0. ]
D = 10 : [0.2 0.2 0.2 0.2 0.2 0.2 0.2 0.2 0.2 0.2]
D = 11 : [0.2 0.2 0.2 0.2 0.2 0.2 0.2 0.2 0.2 0.2]
D = 12 : [0.2 0.2 0.2 0.2 0.2 0.2 0.2 0.2 0.2 0.2]
D = 13 : [0.2 0.2 0.2 0.2 0.2 0.2 0.2 0.2 0.2 0.2]
D = 14 : [0.2 0.2 0.2 0.2 0.2 0.2 0.2 0.2 0.2 0.2]


In [30]:
k = 3
thetas = k*[1] + (n-k)*[0]

for max_distance in range(1, 15):
    perms = generate_kendall_tau_permutations(n, max_distance, distance_dict=distance_dict)
    v_out, p_out, tight_constraints = get_optimal_strategy(thetas, k, perms, lex_order_p=1) 
    print('D =', max_distance,':', np.round(p_out.tolist(), 3))

D = 1 : [1.  1.  0.5 0.5 0.  0.  0.  0.  0.  0. ]
D = 2 : [1. 1. 1. 0. 0. 0. 0. 0. 0. 0.]
D = 3 : [1. 1. 1. 0. 0. 0. 0. 0. 0. 0.]
D = 4 : [1.    0.667 0.667 0.333 0.333 0.    0.    0.    0.    0.   ]
D = 5 : [1.  1.  0.5 0.5 0.  0.  0.  0.  0.  0. ]
D = 6 : [0.857 0.857 0.429 0.429 0.429 0.    0.    0.    0.    0.   ]
D = 7 : [0.6 0.6 0.6 0.6 0.6 0.  0.  0.  0.  0. ]
D = 8 : [0.667 0.583 0.5   0.417 0.333 0.25  0.167 0.083 0.    0.   ]
D = 9 : [0.667 0.667 0.667 0.333 0.333 0.333 0.    0.    0.    0.   ]
D = 10 : [0.6   0.533 0.467 0.4   0.333 0.267 0.2   0.133 0.067 0.   ]
D = 11 : [0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3]
D = 12 : [0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3]
D = 13 : [0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3]
D = 14 : [0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3]


In [33]:
k = 3
thetas = k*[1] + (n-k)*[0]
# thetas += (n - np.arange(n)) / 1000. # make stricly decreasing 

for max_distance in range(1, 15):
    perms = generate_kendall_tau_permutations(n, max_distance, distance_dict=distance_dict)
    v_out, p_out, tight_constraints = get_optimal_strategy(thetas, k, perms, lex_order_p=-1) 
    print('D =', max_distance,':', np.round(p_out.tolist(), 3))

D = 1 : [1.  1.  0.5 0.5 0.  0.  0.  0.  0.  0. ]
D = 2 : [1.  0.5 0.5 0.5 0.5 0.  0.  0.  0.  0. ]
D = 3 : [1. 1. 1. 0. 0. 0. 0. 0. 0. 0.]
D = 4 : [1.    0.667 0.667 0.333 0.333 0.    0.    0.    0.    0.   ]
D = 5 : [0.75 0.75 0.75 0.75 0.   0.   0.   0.   0.   0.  ]
D = 6 : [0.714 0.571 0.571 0.429 0.286 0.286 0.143 0.    0.    0.   ]
D = 7 : [0.6 0.6 0.6 0.6 0.6 0.  0.  0.  0.  0. ]
D = 8 : [0.667 0.583 0.5   0.417 0.333 0.25  0.167 0.083 0.    0.   ]
D = 9 : [0.5 0.5 0.5 0.5 0.5 0.5 0.  0.  0.  0. ]
D = 10 : [0.6   0.533 0.467 0.4   0.333 0.267 0.2   0.133 0.067 0.   ]
D = 11 : [0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3]
D = 12 : [0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3]
D = 13 : [0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3]
D = 14 : [0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3]


In [ ]:
k = 3
thetas = k*[1] + (n-k)*[0]
thetas += (n - np.arange(n)) / 1000. # make stricly decreasing 

for max_distance in range(1, 15):
    perms = generate_kendall_tau_permutations(n, max_distance, distance_dict=distance_dict)
    v_out, p_out, tight_constraints = get_optimal_strategy(thetas, k, perms, lex_order_p=-1) 
    print('D =', max_distance,':', np.round(p_out.tolist(), 3))

D = 1 : [1.  1.  0.5 0.5 0.  0.  0.  0.  0.  0. ]
D = 2 : [1.    1.    0.999 0.001 0.    0.    0.    0.    0.    0.   ]
D = 3 : [1.    0.999 0.998 0.002 0.001 0.    0.    0.    0.    0.   ]
D = 4 : [1.    0.667 0.666 0.334 0.333 0.    0.    0.    0.    0.   ]
D = 5 : [1.    0.999 0.5   0.5   0.001 0.    0.    0.    0.    0.   ]
D = 6 : [0.857 0.856 0.429 0.429 0.428 0.001 0.    0.    0.    0.   ]
D = 7 : [0.601 0.6   0.6   0.599 0.598 0.001 0.001 0.    0.    0.   ]
D = 8 : [0.667 0.583 0.5   0.417 0.333 0.25  0.167 0.083 0.    0.   ]
D = 9 : [0.667 0.666 0.666 0.334 0.333 0.333 0.001 0.    0.    0.   ]


In [8]:
k = 4
thetas = k*[1] + (n-k)*[0]

for max_distance in range(1, 15):
    perms = generate_kendall_tau_permutations(n, max_distance, distance_dict=distance_dict)
    v_out, p_out, tight_constraints = get_optimal_strategy(thetas, k, perms, lex_order_p=True) 
    print('D =', max_distance,':', np.round(p_out.tolist(), 3))

D = 1 : [1.  1.  1.  0.5 0.5 0.  0.  0.  0.  0. ]
D = 2 : [1. 1. 1. 1. 0. 0. 0. 0. 0. 0.]
D = 3 : [1. 1. 1. 1. 0. 0. 0. 0. 0. 0.]
D = 4 : [1.    1.    0.667 0.667 0.333 0.333 0.    0.    0.    0.   ]
D = 5 : [1.  1.  1.  0.5 0.5 0.  0.  0.  0.  0. ]
D = 6 : [1.    0.857 0.714 0.571 0.429 0.286 0.143 0.    0.    0.   ]
D = 7 : [1.    0.857 0.714 0.571 0.429 0.286 0.143 0.    0.    0.   ]
D = 8 : [1. 1. 1. 1. 0. 0. 0. 0. 0. 0.]
D = 9 : [0.889 0.778 0.667 0.556 0.444 0.333 0.222 0.111 0.    0.   ]
D = 10 : [0.889 0.778 0.667 0.556 0.444 0.333 0.222 0.111 0.    0.   ]
D = 11 : [0.8   0.711 0.622 0.533 0.444 0.356 0.267 0.178 0.089 0.   ]
D = 12 : [0.8   0.711 0.622 0.533 0.444 0.356 0.267 0.178 0.089 0.   ]
D = 13 : [0.4 0.4 0.4 0.4 0.4 0.4 0.4 0.4 0.4 0.4]
D = 14 : [0.4 0.4 0.4 0.4 0.4 0.4 0.4 0.4 0.4 0.4]


In [17]:
def solve_lp(theta_perms):
    n = len(theta_perms[0])
    k = sum(theta_perms[0])
    # Decision variables
    v = cp.Variable()
    p = cp.Variable(n, nonneg=True)

    # Constraints: adversary's worst-case selection
    constraints = [v - cp.matmul(theta_p, p) <= 0 for theta_p in theta_perms]

    # Constraints: p sums to k
    constraints.append(cp.sum(p) == k)

    # Constraints: all p_i are in [0, 1]
    constraints += [p[i] <= 1 for i in range(n)]

    # **Step 1: Solve for v* without modifying p**
    problem = cp.Problem(cp.Maximize(v), constraints)
    problem.solve()

    return v.value, p.value

# search every subset of tight constraints to find smallest subset that has same value as original tight constraints 
def find_subset(tight_constraints, v_original):
    n = len(tight_constraints)
    for i in range(1, n+1):
        for subset in itertools.combinations(tight_constraints, i):
            v, p = solve_lp(subset)
            # check infeasible
            if v is None:
                continue
            # value is the same as original LP
            if np.isclose(v, v_original):
                # evaluate p on the original tight constraints
                v_tight = min([np.dot(p, theta_p) for theta_p in tight_constraints])
                if np.isclose(v_tight, v_original):
                    return subset
    return None

In [15]:
k = 3
thetas = k*[1] + (n-k)*[0]
D = 4
perms = generate_kendall_tau_permutations(n, D, distance_dict=distance_dict)
v_out, p_out, tight_constraints = get_optimal_strategy(thetas, k, perms, lex_order_p=True, return_tight_constraints=True) 

In [16]:
len(tight_constraints)

6

In [17]:
find_subset(tight_constraints, v_out)

((0, 1, 1, 1, 0, 0, 0, 0, 0, 0),
 (0, 1, 1, 0, 1, 0, 0, 0, 0, 0),
 (1, 1, 0, 0, 0, 1, 0, 0, 0, 0),
 (1, 0, 0, 1, 1, 0, 0, 0, 0, 0),
 (1, 0, 1, 0, 0, 1, 0, 0, 0, 0))

In [18]:
k = 3
thetas = k*[1] + (n-k)*[0]
D = 5
perms = generate_kendall_tau_permutations(n, D, distance_dict=distance_dict)
v_out, p_out, tight_constraints = get_optimal_strategy(thetas, k, perms, lex_order_p=True, return_tight_constraints=True) 

In [19]:
len(tight_constraints)

8

In [20]:
find_subset(tight_constraints, v_out)

((1, 0, 1, 0, 0, 0, 1, 0, 0, 0),
 (1, 0, 0, 1, 0, 1, 0, 0, 0, 0),
 (0, 1, 0, 1, 1, 0, 0, 0, 0, 0),
 (0, 1, 1, 0, 0, 1, 0, 0, 0, 0))

In [355]:
# for num1 in range(k):
#     for num_unif in range(1, n - num1):
#         if (k - num1) / num_unif > 1:
#             continue
#         p = [1]*num1 + [(k - num1) / num_unif]*num_unif + [0]*(n - num1 - num_unif)
#         # get min over all permutations sigma of p^T * theta_sigma 
#         worst_case_value, _ = evaluate_strategy(s, p, generate_permutations)
#         # print(np.round(p, 3))
#         # print(round(worst_case_value, 3))
#         if np.isclose(worst_case_value, v_out):
#             print(p)
#             break 

### Borda Count

In [7]:
k = 1
# borda count
thetas = n - np.arange(1, n+1)

for max_distance in range(1, 9):
    perms = generate_kendall_tau_permutations(n, max_distance, distance_dict=distance_dict)
    v_out, p_out, tight_constraints = get_optimal_strategy(thetas, k, perms, lex_order_p=1) 
    print('D =', max_distance,':', np.round(p_out.tolist(), 3))

D = 1 : [0.667 0.333 0.    0.    0.    0.    0.    0.    0.    0.   ]
D = 2 : [0.5   0.333 0.167 0.    0.    0.    0.    0.    0.    0.   ]
D = 3 : [0.4 0.3 0.2 0.1 0.  0.  0.  0.  0.  0. ]
D = 4 : [0.4   0.32  0.224 0.056 0.    0.    0.    0.    0.    0.   ]
D = 5 : [0.354 0.295 0.246 0.101 0.004 0.    0.    0.    0.    0.   ]
D = 6 : [0.309 0.265 0.227 0.155 0.044 0.    0.    0.    0.    0.   ]
D = 7 : [0.285 0.249 0.218 0.175 0.073 0.    0.    0.    0.    0.   ]
D = 8 : [0.265 0.235 0.224 0.173 0.086 0.017 0.    0.    0.    0.   ]


In [8]:
k = 1
# borda count
thetas = n - np.arange(1, n+1)

for max_distance in range(1, 9):
    perms = generate_kendall_tau_permutations(n, max_distance, distance_dict=distance_dict)
    v_out, p_out, tight_constraints = get_optimal_strategy(thetas, k, perms, lex_order_p=-1) 
    print('D =', max_distance,':', np.round(p_out.tolist(), 3))

D = 1 : [0.667 0.333 0.    0.    0.    0.    0.    0.    0.    0.   ]
D = 2 : [0.5   0.333 0.167 0.    0.    0.    0.    0.    0.    0.   ]
D = 3 : [0.4 0.3 0.2 0.1 0.  0.  0.  0.  0.  0. ]
D = 4 : [0.4   0.32  0.224 0.056 0.    0.    0.    0.    0.    0.   ]
D = 5 : [0.354 0.295 0.246 0.101 0.004 0.    0.    0.    0.    0.   ]
D = 6 : [0.309 0.265 0.227 0.155 0.044 0.    0.    0.    0.    0.   ]
D = 7 : [0.285 0.249 0.218 0.175 0.073 0.    0.    0.    0.    0.   ]
D = 8 : [0.265 0.235 0.224 0.173 0.086 0.017 0.    0.    0.    0.   ]


In [22]:
k = 2
# borda count
thetas = n - np.arange(1, n+1)

for max_distance in range(1, 9):
    perms = generate_kendall_tau_permutations(n, max_distance, distance_dict=distance_dict)
    v_out, p_out, tight_constraints = get_optimal_strategy(thetas, k, perms, lex_order_p=1) 
    print('D =', max_distance,':', np.round(p_out.tolist(), 3))

D = 1 : [1.    0.667 0.333 0.    0.    0.    0.    0.    0.    0.   ]
D = 2 : [1.    0.667 0.333 0.    0.    0.    0.    0.    0.    0.   ]
D = 3 : [0.8 0.6 0.4 0.2 0.  0.  0.  0.  0.  0. ]
D = 4 : [0.8   0.64  0.448 0.112 0.    0.    0.    0.    0.    0.   ]
D = 5 : [0.708 0.59  0.492 0.202 0.008 0.    0.    0.    0.    0.   ]
D = 6 : [0.618 0.529 0.454 0.311 0.088 0.    0.    0.    0.    0.   ]
D = 7 : [0.569 0.498 0.436 0.351 0.146 0.    0.    0.    0.    0.   ]
D = 8 : [0.529 0.471 0.449 0.345 0.172 0.034 0.    0.    0.    0.   ]


In [9]:
k = 2
# borda count
thetas = n - np.arange(1, n+1)

for max_distance in range(1, 9):
    perms = generate_kendall_tau_permutations(n, max_distance, distance_dict=distance_dict)
    v_out, p_out, tight_constraints = get_optimal_strategy(thetas, k, perms, lex_order_p=-1) 
    print('D =', max_distance,':', np.round(p_out.tolist(), 3))

D = 1 : [1.    0.667 0.333 0.    0.    0.    0.    0.    0.    0.   ]
D = 2 : [1.    0.667 0.333 0.    0.    0.    0.    0.    0.    0.   ]
D = 3 : [0.8 0.6 0.4 0.2 0.  0.  0.  0.  0.  0. ]
D = 4 : [0.8   0.64  0.448 0.112 0.    0.    0.    0.    0.    0.   ]
D = 5 : [0.708 0.59  0.492 0.202 0.008 0.    0.    0.    0.    0.   ]
D = 6 : [0.618 0.529 0.454 0.311 0.088 0.    0.    0.    0.    0.   ]
D = 7 : [0.569 0.498 0.436 0.351 0.146 0.    0.    0.    0.    0.   ]
D = 8 : [0.529 0.471 0.449 0.345 0.172 0.034 0.    0.    0.    0.   ]


In [23]:
k = 3
# borda count
thetas = n - np.arange(1, n+1)

for max_distance in range(1, 9):
    perms = generate_kendall_tau_permutations(n, max_distance, distance_dict=distance_dict)
    v_out, p_out, tight_constraints = get_optimal_strategy(thetas, k, perms, lex_order_p=True) 
    print('D =', max_distance,':', np.round(p_out.tolist(), 3))

D = 1 : [1.    1.    0.667 0.333 0.    0.    0.    0.    0.    0.   ]
D = 2 : [1.    1.    0.667 0.333 0.    0.    0.    0.    0.    0.   ]
D = 3 : [1.  0.8 0.6 0.4 0.2 0.  0.  0.  0.  0. ]
D = 4 : [1.   0.8  0.64 0.36 0.2  0.   0.   0.   0.   0.  ]
D = 5 : [1.    0.833 0.694 0.394 0.079 0.    0.    0.    0.    0.   ]
D = 6 : [0.926 0.794 0.681 0.466 0.132 0.    0.    0.    0.    0.   ]
D = 7 : [0.854 0.747 0.654 0.526 0.219 0.    0.    0.    0.    0.   ]
D = 8 : [0.794 0.706 0.673 0.518 0.258 0.05  0.    0.    0.    0.   ]


In [10]:
k = 3
# borda count
thetas = n - np.arange(1, n+1)

for max_distance in range(1, 9):
    perms = generate_kendall_tau_permutations(n, max_distance, distance_dict=distance_dict)
    v_out, p_out, tight_constraints = get_optimal_strategy(thetas, k, perms, lex_order_p=-1) 
    print('D =', max_distance,':', np.round(p_out.tolist(), 3))

D = 1 : [1.    1.    0.667 0.333 0.    0.    0.    0.    0.    0.   ]
D = 2 : [1.    1.    0.667 0.333 0.    0.    0.    0.    0.    0.   ]
D = 3 : [1.  0.8 0.6 0.4 0.2 0.  0.  0.  0.  0. ]
D = 4 : [1.   0.8  0.64 0.36 0.2  0.   0.   0.   0.   0.  ]
D = 5 : [1.    0.833 0.694 0.394 0.079 0.    0.    0.    0.    0.   ]
D = 6 : [0.926 0.794 0.681 0.466 0.132 0.    0.    0.    0.    0.   ]
D = 7 : [0.854 0.747 0.654 0.526 0.219 0.    0.    0.    0.    0.   ]
D = 8 : [0.794 0.706 0.673 0.518 0.258 0.05  0.    0.    0.    0.   ]


In [24]:
k = 4
# borda count
thetas = n - np.arange(1, n+1)

for max_distance in range(1, 9):
    perms = generate_kendall_tau_permutations(n, max_distance, distance_dict=distance_dict)
    v_out, p_out, tight_constraints = get_optimal_strategy(thetas, k, perms, lex_order_p=1) 
    print('D =', max_distance,':', np.round(p_out.tolist(), 3))

D = 1 : [1.    1.    1.    0.667 0.333 0.    0.    0.    0.    0.   ]
D = 2 : [1.    1.    1.    0.667 0.333 0.    0.    0.    0.    0.   ]
D = 3 : [1.  1.  0.8 0.6 0.4 0.2 0.  0.  0.  0. ]
D = 4 : [1.   1.   0.8  0.64 0.36 0.2  0.   0.   0.   0.  ]
D = 5 : [1.    1.    0.833 0.694 0.306 0.167 0.    0.    0.    0.   ]
D = 6 : [1.    1.    0.857 0.735 0.343 0.065 0.    0.    0.    0.   ]
D = 7 : [1.    0.918 0.804 0.746 0.399 0.133 0.    0.    0.    0.   ]
D = 8 : [1.    0.889 0.854 0.683 0.407 0.167 0.    0.    0.    0.   ]


In [11]:
k = 4
# borda count
thetas = n - np.arange(1, n+1)

for max_distance in range(1, 9):
    perms = generate_kendall_tau_permutations(n, max_distance, distance_dict=distance_dict)
    v_out, p_out, tight_constraints = get_optimal_strategy(thetas, k, perms, lex_order_p=-1) 
    print('D =', max_distance,':', np.round(p_out.tolist(), 3))

D = 1 : [1.    1.    1.    0.667 0.333 0.    0.    0.    0.    0.   ]
D = 2 : [1.    1.    1.    0.667 0.333 0.    0.    0.    0.    0.   ]
D = 3 : [1.  1.  0.8 0.6 0.4 0.2 0.  0.  0.  0. ]
D = 4 : [1.   1.   0.8  0.64 0.36 0.2  0.   0.   0.   0.  ]
D = 5 : [1.    1.    0.833 0.694 0.306 0.167 0.    0.    0.    0.   ]
D = 6 : [1.    1.    0.857 0.735 0.343 0.065 0.    0.    0.    0.   ]
D = 7 : [1.    0.918 0.804 0.746 0.399 0.133 0.    0.    0.    0.   ]
D = 8 : [1.    0.889 0.854 0.683 0.407 0.167 0.    0.    0.    0.   ]


In [38]:
k = 3
thetas = n - np.arange(1, n+1)
D = 5
perms = generate_kendall_tau_permutations(n, D, distance_dict=distance_dict)
v_out, p_out, tight_constraints = get_optimal_strategy(thetas, k, perms, lex_order_p=1, return_tight_constraints=True) 

In [39]:
# for two sequences find permutation that makes first sequence into the second
def find_permutation(seq1, seq2):
    n = len(seq1)
    perm = [seq2.index(seq1[i]) for i in range(n)]
    return perm

In [40]:
sorted([find_permutation(seq, sorted(seq, reverse=True)) for seq in tight_constraints])

[[0, 1, 7, 2, 3, 4, 5, 6, 8, 9],
 [0, 6, 1, 2, 3, 4, 5, 7, 8, 9],
 [1, 2, 3, 4, 5, 0, 6, 7, 8, 9],
 [1, 2, 3, 5, 0, 4, 6, 7, 8, 9],
 [5, 0, 1, 2, 3, 4, 6, 7, 8, 9]]

In [41]:
k = 3
thetas = k*[1] + (n-k)*[0]
thetas += (n - np.arange(n)) / 1000. # make stricly decreasing

D = 5
perms = generate_kendall_tau_permutations(n, D, distance_dict=distance_dict)
v_out, p_out, tight_constraints = get_optimal_strategy(thetas, k, perms, lex_order_p=1, return_tight_constraints=True) 

In [42]:
sorted([find_permutation(seq, sorted(seq, reverse=True)) for seq in tight_constraints])

[[0, 3, 4, 1, 5, 2, 6, 7, 8, 9],
 [0, 4, 1, 5, 2, 3, 6, 7, 8, 9],
 [0, 5, 1, 3, 2, 4, 6, 7, 8, 9],
 [1, 3, 2, 4, 0, 5, 6, 7, 8, 9],
 [2, 3, 0, 4, 1, 5, 6, 7, 8, 9],
 [3, 0, 1, 4, 5, 2, 6, 7, 8, 9],
 [3, 0, 4, 1, 2, 5, 6, 7, 8, 9]]

## Simulations with Bounded Footrule Distance

In [222]:
n = 10
k = 3
s =k*[1] + (n-k)*[0] # top k

In [223]:
distance_dict = get_all_spearmans_footrule_distances(n)

In [227]:
for max_distance in range(1, 14):
    perms = generate_spearmans_footrule_permutations(n, 2*max_distance, distance_dict=distance_dict)
    v_out, p_out, tight_constraints = get_optimal_strategy(s, k, perms, lex_order_p=True) 
    print('D =', 2*max_distance,':', np.round(p_out.tolist(), 3))

D = 2 : [1.  1.  0.5 0.5 0.  0.  0.  0.  0.  0. ]
D = 4 : [1. 1. 1. 0. 0. 0. 0. 0. 0. 0.]
D = 6 : [1. 1. 1. 0. 0. 0. 0. 0. 0. 0.]
D = 8 : [1.    0.667 0.667 0.333 0.333 0.    0.    0.    0.    0.   ]
D = 10 : [1.  1.  0.5 0.5 0.  0.  0.  0.  0.  0. ]
D = 12 : [0.857 0.857 0.429 0.429 0.429 0.    0.    0.    0.    0.   ]
D = 14 : [0.6 0.6 0.6 0.6 0.6 0.  0.  0.  0.  0. ]
D = 16 : [0.667 0.583 0.5   0.417 0.333 0.25  0.167 0.083 0.    0.   ]
D = 18 : [0.667 0.667 0.667 0.333 0.333 0.333 0.    0.    0.    0.   ]
D = 20 : [0.6   0.533 0.467 0.4   0.333 0.267 0.2   0.133 0.067 0.   ]
D = 22 : [0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3]
D = 24 : [0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3]
D = 26 : [0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3]


In [382]:
n = 10
k = 4
s =k*[1] + (n-k)*[0] # top k

print(s)


[1, 1, 1, 1, 0, 0, 0, 0, 0, 0]
D = 1 : [1.  1.  1.  0.5 0.5 0.  0.  0.  0.  0. ]
D = 2 : [1. 1. 1. 1. 0. 0. 0. 0. 0. 0.]
D = 3 : [1. 1. 1. 1. 0. 0. 0. 0. 0. 0.]
D = 4 : [1.    1.    0.667 0.667 0.333 0.333 0.    0.    0.    0.   ]
D = 5 : [1.  1.  1.  0.5 0.5 0.  0.  0.  0.  0. ]
D = 6 : [1.    0.857 0.714 0.571 0.429 0.286 0.143 0.    0.    0.   ]
D = 7 : [1.    0.857 0.714 0.571 0.429 0.286 0.143 0.    0.    0.   ]
D = 8 : [1. 1. 1. 1. 0. 0. 0. 0. 0. 0.]


### Simulations with Bounded L_infinity distance

In [4]:
n = 10

In [5]:
distance_dict = get_all_linf_distances(n)

In [6]:
k = 1
s = n - np.arange(n) - 1 # borda count
for D in range(1, 5):
    perms = generate_linf_permutations(n, D, distance_dict)
    v_out, p_out, tight_constraints = get_optimal_strategy(s, k, perms, lex_order_p=True) 
    print('D =', D,':', np.round(p_out.tolist(), 3))

D = 1 : [0.667 0.333 0.    0.    0.    0.    0.    0.    0.    0.   ]
D = 2 : [0.414 0.276 0.207 0.103 0.    0.    0.    0.    0.    0.   ]
D = 3 : [0.377 0.189 0.151 0.126 0.094 0.063 0.    0.    0.    0.   ]
D = 4 : [0.315 0.158 0.126 0.108 0.095 0.079 0.068 0.053 0.    0.   ]


In [239]:
evaluate_strategy(s, [12./29, 8./29, 6./29, 3./29] + [0]*6, lambda _: generate_linf_permutations(n, 2, distance_dict))

(8.241379310344827, (0, 3, 4, 1, 2, 5, 6, 7, 8, 9))

In [240]:
evaluate_strategy(s, [8./17, 4./17, 3./17, 2./17] + [0]*6, lambda _: generate_linf_permutations(n, 2, distance_dict))

(8.235294117647058, (1, 0, 4, 5, 2, 3, 6, 7, 8, 9))

In [245]:
import math

def get_possible_worst_case_permutations(D):
    # assume n>=3d
    n = 3*D
    thetas = n - np.arange(n) - 1
        # solve for k=1, all ranks can be shifted by at most D
    supp = 2*D
    n = len(thetas)
    perm0 = np.hstack((thetas[D:2*D], thetas[:D])).tolist() # initial tight permuatation is (D, D+1, ..., 2D-1, 0, 1, ..., D-1)
    perms = [perm0]
    for i in range(0, supp-1):
        for j in range(i+1, supp):
            # perm[i] could be any value in [i-D, i+D] - {perm0 without i,j}
            # perm[j] could be any value in [j-D, j+D] - {perm0 without i,j}
            perm = perm0.copy()
            perm.pop(i)
            perm.pop(j-1)
            for val_i in range(max(i-D, 0), min(i+D+1, n)):
                for val_j in range(max(j-D, 0), min(j+D+1, n)):
                    # values are valid to choose
                    if val_i == val_j or thetas[val_i] in perm or thetas[val_j] in perm:
                        continue
                    # if val_i <= i and val_j <= i then this will not be a worst case permutation
                    if val_i <= i and val_j <= j:
                        continue
                    perm.insert(i, thetas[val_i])
                    perm.insert(j, thetas[val_j])
                    # all values of thetas[0:D] must be contained in perm to be valid
                    if set(thetas[:D]).issubset(set(perm)):
                        perms.append(tuple(perm))
                    perm.pop(i)
                    perm.pop(j-1)
    perms = list(set(map(tuple, perms)))
    # iterate over perms to check if one perm "dominates" another and eliminate dominated perm
    # perm2 dominates perm1 if all values of perm2 are less than or equal to perm1 => perm1 cant be worst case
    out = []
    for perm1 in perms:
        dominated = False
        for perm2 in perms:
            if all(perm2[i] <= perm1[i] for i in range(supp)) and perm1 != perm2:
                dominated = True
                break
        if not dominated:
            out.append(perm1)
    
    # remove perm0 from perms
    out = [perm for perm in out if perm != tuple(perm0)]
    # reverse Borda count to get back ranks
    out = [tuple(n-p-1 for p in perm) for perm in out]
    return tuple(n-p-1 for p in perm0), sorted(out, reverse=True)

def get_constraint(thetas, perm0, perm):
    # perm0 and perm should differ in 2 places, get the two places
    supp = len(perm0)
    # get the two places where perm0 and perm differ
    i = None
    j = None
    for k in range(supp):
        if perm0[k] != perm[k]:
            if i is None:
                i = k
            elif j is None:
                j = k
            else:
                raise ValueError("More than 2 places differ")
    if i is None or j is None:
        raise ValueError("Less than 2 places differ")
    ratio = (thetas[perm[j]] - thetas[perm0[j]]) / (thetas[perm0[i]] - thetas[perm[i]])
    return (i, j, ratio)

def solve_efficient_linf(thetas, D):
    perm0, perms = get_possible_worst_case_permutations(D)
    # sort perms in decreasing order of L1 distance from (0,..,2D-1)
    perms = sorted(perms, key=lambda x: sum(abs(x[i] - i) for i in range(len(x))), reverse=True)
    return perms, [get_constraint(thetas, perm0, perm) for perm in perms]


In [244]:
constraints

([(2, 3, 1.2),
  (0, 5, 6.0),
  (2, 4, 1.5),
  (1, 5, 3.0),
  (1, 3, 1.5),
  (0, 4, 3.0),
  (2, 5, 2.0),
  (1, 4, 2.0),
  (0, 3, 2.0)],
 6)

In [248]:
perms, constraints = solve_efficient_linf(s, 3)
# add 1 to all the ranks 
perms = [tuple(p+1 for p in perm) for perm in perms]
constraints = [(i+1, j+1, ratio) for i, j, ratio in constraints]
perms, constraints

([(4, 5, 1, 7, 2, 3),
  (3, 5, 6, 1, 2, 9),
  (4, 5, 2, 1, 8, 3),
  (4, 3, 6, 1, 2, 9),
  (4, 1, 6, 7, 2, 3),
  (2, 5, 6, 1, 8, 3),
  (4, 5, 3, 1, 2, 9),
  (4, 2, 6, 1, 8, 3),
  (1, 5, 6, 7, 2, 3)],
 [(3, 4, 1.2),
  (1, 6, 6.0),
  (3, 5, 1.5),
  (2, 6, 3.0),
  (2, 4, 1.5),
  (1, 5, 3.0),
  (3, 6, 2.0),
  (2, 5, 2.0),
  (1, 4, 2.0)])

In [228]:
solve_efficient_linf(s,4)

IndexError: index 11 is out of bounds for axis 0 with size 10

In [27]:
D = 3
perms = generate_linf_permutations(n, D, distance_dict)
v_out, p_out, tight_constraints = get_optimal_strategy(s, 1, perms, return_tight_constraints=True) 

In [44]:
supp = np.where(p_out > 1e-6)[0]
vals = set([tuple(t[i] for i in supp) for t in tight_constraints])
perms = [tuple(n - v for v in t) for t in vals]
sorted(perms, reverse=True)

[(4, 5, 6, 1, 2, 3),
 (4, 5, 1, 7, 2, 3),
 (4, 3, 6, 1, 2, 9),
 (4, 3, 1, 7, 2, 9),
 (4, 2, 6, 1, 8, 3),
 (4, 2, 1, 7, 8, 3),
 (4, 1, 6, 7, 2, 3),
 (3, 5, 6, 1, 2, 9),
 (3, 5, 1, 7, 2, 9),
 (3, 2, 6, 1, 8, 9),
 (3, 2, 1, 7, 8, 9),
 (3, 1, 6, 7, 2, 9)]

In [46]:
P0 = (4,5,6,1,2,3)
[p for p in perms if sum(p[i] - P0[i] != 0 for i in range(len(P0))) == 2]

[(3, 5, 6, 1, 2, 9),
 (4, 5, 1, 7, 2, 3),
 (4, 2, 6, 1, 8, 3),
 (4, 1, 6, 7, 2, 3),
 (4, 3, 6, 1, 2, 9)]

In [28]:
np.round(p_out, 5)

array([0.37736, 0.18868, 0.15094, 0.12579, 0.09434, 0.06289, 0.     ,
       0.     , 0.     , 0.     ])

In [20]:
p_out[0] / p_out[2]

2.0000000000239964

In [ ]:
tight_constraints

In [7]:
k = 2
s = n - np.arange(n) - 1 # borda count
for D in range(1, 5):
    perms = generate_linf_permutations(n, D, distance_dict)
    v_out, p_out, tight_constraints = get_optimal_strategy(s, k, perms, lex_order_p=True) 
    print('D =', D,':', np.round(p_out.tolist(), 3))

D = 1 : [1.   0.75 0.25 0.   0.   0.   0.   0.   0.   0.  ]
D = 2 : [0.828 0.552 0.414 0.207 0.    0.    0.    0.    0.    0.   ]
D = 3 : [0.755 0.377 0.302 0.252 0.189 0.126 0.    0.    0.    0.   ]
D = 4 : [0.63  0.315 0.252 0.216 0.189 0.158 0.135 0.105 0.    0.   ]


In [89]:
k = 5
s = k*[1] + (n-k)*[0] # top k
for D in range(1, 6):
    perms = generate_linf_permutations(n, D, distance_dict)
    v_out, p_out, tight_constraints = get_optimal_strategy(s, k, perms, lex_order_p=True) 
    print('D =', D,':', np.round(p_out.tolist(), 3))

D = 1 : [1.  1.  1.  1.  0.5 0.5 0.  0.  0.  0. ]
D = 2 : [1.  1.  1.  0.5 0.5 0.5 0.5 0.  0.  0. ]
D = 3 : [1.  1.  0.5 0.5 0.5 0.5 0.5 0.5 0.  0. ]
D = 4 : [1.  0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0. ]
D = 5 : [0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]


In [90]:
k = 3
s = k*[1] + (n-k)*[0] # top k
for D in range(1, 6):
    perms = generate_linf_permutations(n, D, distance_dict)
    v_out, p_out, tight_constraints = get_optimal_strategy(s, k, perms, lex_order_p=True) 
    print('D =', D,':', np.round(p_out.tolist(), 3))

D = 1 : [1.  1.  0.5 0.5 0.  0.  0.  0.  0.  0. ]
D = 2 : [1.  0.5 0.5 0.5 0.5 0.  0.  0.  0.  0. ]
D = 3 : [0.5 0.5 0.5 0.5 0.5 0.5 0.  0.  0.  0. ]
D = 4 : [0.429 0.429 0.429 0.429 0.429 0.429 0.429 0.    0.    0.   ]
D = 5 : [0.375 0.375 0.375 0.375 0.375 0.375 0.375 0.375 0.    0.   ]


In [78]:
theta_p

[7, 6, 9, 8, 3, 2, 5, 4, 0, 1]

In [91]:
perm1 = (3,4,1,2,7,8,5,6,10,9)
perm1 = tuple(s-1 for s in perm1)
print(perm1)

perm2 = (2,1,5,6,3,4,9,10,7,8)
perm2 = tuple(s-1 for s in perm2)
print(perm2)

perm3 = (1,4,5,2,3,8,9,6,7,10)
perm3 = tuple(s-1 for s in perm3)
print(perm3)

(2, 3, 0, 1, 6, 7, 4, 5, 9, 8)
(1, 0, 4, 5, 2, 3, 8, 9, 6, 7)
(0, 3, 4, 1, 2, 7, 8, 5, 6, 9)


In [92]:
 # Decision variables
v = cp.Variable()
p = cp.Variable(n, nonneg=True)

theta_perms = [tuple(s[r] for r in perm) for perm in [perm1, perm2, perm3]]

# Constraints: adversary's worst-case selection
constraints = [v - cp.matmul(theta_p, p) <= 0 for theta_p in theta_perms]

# Constraints: p sums to k
constraints.append(cp.sum(p) == k)

# Constraints: all p_i are in [0, 1]
constraints += [p[i] <= 1 for i in range(n)]

# p_i are non-increasing 
constraints += [p[i] >= p[i+1] for i in range(n-1)]

# **Step 1: Solve for v* without modifying p**
problem = cp.Problem(cp.Maximize(v), constraints)
problem.solve()
v_star = v.value  # Store optimal v

In [93]:
np.round(p.value, 3)

array([1. , 1. , 0.5, 0.5, 0. , 0. , 0. , 0. , 0. , 0. ])

In [94]:
v.value

array(21.5)

In [236]:
k = 2
s = n - np.arange(n)  # borda count
D = 2
perms = generate_linf_permutations(n, D, distance_dict)
v_out, p_out, tight_constraints = get_optimal_strategy(s, k, perms, lex_order_p=0, return_tight_constraints=True)

In [ ]:
k = 5
s = n - np.arange(n)  # borda count
for D in range(1, 4):
    perms = generate_linf_permutations(n, D, distance_dict)
    v_out, p_out, tight_constraints = get_optimal_strategy(s, k, perms, lex_order_p=True) 
    print('D =', D,':', np.round(p_out.tolist(), 3))

D = 1 : [1.   1.   1.   1.   0.75 0.25 0.   0.   0.   0.  ]
D = 2 : [1.    1.    1.    0.875 0.625 0.375 0.125 0.    0.    0.   ]
D = 3 : [1.    1.    0.9   0.75  0.583 0.417 0.25  0.1   0.    0.   ]


In [ ]:
k = 3
s = k*[1] + (n-k)*[0] # top k
for D in range(1, 6):
    perms = generate_linf_permutations(n, D, distance_dict)
    v_out, p_out, tight_constraints = get_optimal_strategy(s, k, perms, lex_order_p=True) 
    print('D =', D,':', np.round(p_out.tolist(), 3))

In [82]:
k = 6
s = k*[1] + (n-k)*[0] # top k
for D in range(1, 6):
    perms = generate_mD_insertion_permutations(n, 3, D)
    v_out, p_out, tight_constraints = get_optimal_strategy(s, k, perms, lex_order_p=True, return_tight_constraints=True) 
    print('D =', D,':', np.round(p_out.tolist(), 3))

D = 1 : [1.   1.   1.   1.   0.75 0.5  0.5  0.25 0.   0.  ]
D = 2 : [1.  1.  1.  1.  0.5 0.5 0.5 0.5 0.  0. ]
D = 3 : [1.    1.    1.    0.429 0.429 0.429 0.429 0.429 0.429 0.429]
D = 4 : [1.    1.    1.    1.    0.333 0.333 0.333 0.333 0.333 0.333]
D = 5 : [1.    0.556 0.556 0.556 0.556 0.556 0.556 0.556 0.556 0.556]


In [84]:
perms = generate_mD_insertion_permutations(n, 3, 1)

In [85]:
v_out, p_out, tight_constraints = get_optimal_strategy(s, k, perms, lex_order_p=True, return_tight_constraints=True)

In [86]:
tight_constraints

[(1, 1, 1, 1, 1, 0, 0, 0, 1, 0),
 (1, 1, 1, 1, 0, 0, 1, 1, 0, 0),
 (1, 1, 1, 0, 1, 1, 1, 0, 0, 0),
 (1, 1, 1, 1, 0, 1, 0, 1, 0, 0)]

## MISC: Sampling

In [29]:
'''
Sample a fixed sample size with unequal probabilities without replacement.
Input parameters:
    p: a vector of probabilities for each element in the population (size N). The sum of p should equal the desired sample size (k).
    eps: a small value to check for missing values in p.
    max_iter: maximum number of iterations to run the algorithm.
Output:
    s: a vector of indicators for each element in the population for whether that element was sampled.
'''
def sampford_sampling(p, eps=1e-6, max_iter=500):
    # From Sampford, M. R. (1967). Sampling from a finite population with unequal probabilities without replacement.
    # https://academic.oup.com/biomet/article-abstract/54/3-4/499/230469?redirectedFrom=fulltext 
    # adopted from R package sampling
    def as_int(n):
        return int(round(n))

    if np.any(np.isnan(p)):
        raise ValueError("There are missing values in the p vector")
    
    k = as_int(np.sum(p))
    valid_mask = (p > eps) & (p < 1 - eps)
    pb = p[valid_mask]
    
    if len(pb) < 1:
        raise ValueError("The p vector has all elements outside of the range [eps, 1-eps]")
    
    N = len(pb)
    s = np.copy(p)
    sb = np.full(N, 2)
    y = pb / (1 - pb)
    y /= np.sum(y)
    step = 0
    
    while np.sum(sb <= 1) != N and step <= max_iter:
        sb = np.random.multinomial(1, pb / np.sum(pb)) + np.random.multinomial(as_int(k - 1), y)
        step += 1
    
    if np.sum(sb <= 1) == N:
        s[valid_mask] = sb
    else:
        raise RuntimeError("Too many iterations. The algorithm was stopped.")
    
    return s